In [7]:
import time
import os
from sagemaker import get_execution_role, session
import boto3

region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

In [8]:
import time
model_package_group_name = "scikit-iris-detector-" + str(round(time.time()))
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}

create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws-cn:sagemaker:cn-north-1:876820548815:model-package-group/scikit-iris-detector-1636957027


In [11]:
modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": '390948362332.dkr.ecr.cn-north-1.amazonaws.com.cn/xgboost:1',
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }
# Specify the model source
model_url = "s3://sagemaker-cn-north-1-876820548815/bootcamp-xgboost-churn/output/xgboost-2020-06-10-02-54-55-028/output/model.tar.gz"

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

In [12]:
create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws-cn:sagemaker:cn-north-1:876820548815:model-package/scikit-iris-detector-1636957027/1


In [13]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'scikit-iris-detector-1636957027',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws-cn:sagemaker:cn-north-1:876820548815:model-package/scikit-iris-detector-1636957027/1',
   'ModelPackageDescription': 'Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)',
   'CreationTime': datetime.datetime(2021, 11, 15, 6, 17, 37, 225000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': '2af14f47-7be1-4b63-8758-34f35817321e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2af14f47-7be1-4b63-8758-34f35817321e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '446',
   'date': 'Mon, 15 Nov 2021 06:18:02 GMT'},
  'RetryAttempts': 0}}

In [14]:
ModelPackageName = sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"][0]["ModelPackageArn"]
sm_client.describe_model_package(ModelPackageName=ModelPackageName)

{'ModelPackageGroupName': 'scikit-iris-detector-1636957027',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws-cn:sagemaker:cn-north-1:876820548815:model-package/scikit-iris-detector-1636957027/1',
 'ModelPackageDescription': 'Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)',
 'CreationTime': datetime.datetime(2021, 11, 15, 6, 17, 37, 225000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '390948362332.dkr.ecr.cn-north-1.amazonaws.com.cn/xgboost:1',
    'ImageDigest': 'sha256:a219d35e7bc4158e972b8e6fa18028b05bdc97a9fe8da2ba16f7e3dc1bc685b4',
    'ModelDataUrl': 's3://sagemaker-cn-north-1-876820548815/bootcamp-xgboost-churn/output/xgboost-2020-06-10-02-54-55-028/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': False,


In [15]:
model_package_update_input_dict = {
    "ModelPackageArn" : ModelPackageName,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

In [16]:
sm_client.describe_model_package(ModelPackageName=ModelPackageName)

{'ModelPackageGroupName': 'scikit-iris-detector-1636957027',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws-cn:sagemaker:cn-north-1:876820548815:model-package/scikit-iris-detector-1636957027/1',
 'ModelPackageDescription': 'Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)',
 'CreationTime': datetime.datetime(2021, 11, 15, 6, 17, 37, 225000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '390948362332.dkr.ecr.cn-north-1.amazonaws.com.cn/xgboost:1',
    'ImageDigest': 'sha256:a219d35e7bc4158e972b8e6fa18028b05bdc97a9fe8da2ba16f7e3dc1bc685b4',
    'ModelDataUrl': 's3://sagemaker-cn-north-1-876820548815/bootcamp-xgboost-churn/output/xgboost-2020-06-10-02-54-55-028/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': False,


In [17]:
from sagemaker import ModelPackage

model_package_arn = ModelPackageName
model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=sagemaker_session)
model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

NameError: name 'sagemaker_session' is not defined